Consider the following data:
$$ X = \begin{bmatrix} 1 & 1 & 2 \\ 1 & 2 & 1 \\ 1 & 1 & 1 \end{bmatrix}, \quad y = \begin{bmatrix} 11 \\ 10 \\ 8 \end{bmatrix}$$
where $X$ is the data matrix and $y$ contains the labels.

We want to find the parameter vector 
$$\beta = \begin{bmatrix} \beta_1 & \beta_2 & \beta_3 \end{bmatrix}^T$$ 
that minimizes the loss over all instances $x_i$ (the i-th row of the matrix X):
$$ L(X, \beta, y) = \sum_{i=1}^3(\beta^Tx_i - y_i)^2$$

- Explain the differences between the classical gradient method (GD) and the stochastic gradient method (SGD).

    The classical GD method is an iterative optimization (minimization/maximization) method that uses the gradient of the function as the direction (for maximization problems, the opposite direction for minimization ones) toward which to update the approximate solution: the gradient is computed by approximating its value by using the whole dataset available at each step.

    On the other hand, in the SGD we approximate the gradient by only using a subset of points of the dataset, chosen randomly. It can be proven that the SGD yelds the same results as the classical GD, but is computationally more efficient, since we use less data to approximate the gradient of the function at each step.

    There are many types of SGD: with or without replacement of the subset of points of the dataset used at each iteration, using only one point or a batch of points as subset, introducing the "moment" of the iteration, where we also consider the history of the past directions to update the next, and many more.


- Perform two epochs using SGD with a step size $\eta = 0.1$ and report the errors and the total loss after each epoch; use the initial guess $\beta = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}^T$ . (Run through the instances in order instead of performing a random selection.).

Setup:

In [67]:
import numpy as np
import jax
import jax.numpy as jnp


def loss(X, beta, y):
    return jnp.sum(jnp.square(X @ beta - y))


X = np.array([[1.0, 1.0, 2.0], [1.0, 2.0, 1.0], [1.0, 1.0, 1.0]])
y = np.array([[11.0, 10.0, 8.0]]).T

loss_jit = jax.jit(loss)
dloss_jit = jax.jit(jax.grad(loss, argnums=1))

GD result:

In [103]:
beta = np.array([[1.0, 1.0, 1.0]]).T
learning_rate = 0.1
n_epochs = 2

print(loss_jit(X, beta, y))
for epoch in range(n_epochs):
    grad = dloss_jit(X, beta, y)
    beta -= learning_rate * grad

print(loss_jit(X, beta, y))
print(beta)

110.0
1113.0244
[[-1.8000007]
 [-3.040001 ]
 [-2.6800013]]


As we can see, with the classical GD the method is diverging. This is probably due to the learning rate too high (by reducing it to $\eta = 0.01$ we obtain convergence).

In [102]:
beta = np.array([[1.0, 1.0, 1.0]]).T
learning_rate = 0.01
n_epochs = 2

print(loss_jit(X, beta, y))
for epoch in range(n_epochs):
    grad = dloss_jit(X, beta, y)
    beta -= learning_rate * grad

print(loss_jit(X, beta, y))
print(beta)

110.0
30.156832
[[1.62  ]
 [1.8236]
 [1.8632]]


Let's try with the SGD:

In [101]:
beta = np.array([[1.0, 1.0, 1.0]]).T
learning_rate = 0.1
n_epochs = 2

print(loss_jit(X, beta, y))
for epoch in range(n_epochs):
    i = epoch % X.shape[0]

    grad = dloss_jit(X[i], beta, y[i])
    beta -= learning_rate * grad

print(loss_jit(X, beta, y))
print(beta)

110.0
0.24000011
[[2.2]
 [2. ]
 [3.6]]


This time it's converging. Let's try adding a dynamic learning rate:

In [100]:
beta = np.array([[1.0, 1.0, 1.0]]).T
learning_rate_max = 0.1
learning_rate_min = 0.001
learning_rate_decay = 5000
n_epochs = 2

print(loss_jit(X, beta, y))
for epoch in range(n_epochs):
    i = np.random.choice(X.shape[0], 2)
    learning_rate = max(
        learning_rate_min, learning_rate_max * (1 - epoch / learning_rate_decay)
    )

    grad = dloss_jit(X[i], beta, y[i])
    beta -= learning_rate * grad

print(loss_jit(X, beta, y))
print(beta)

110.0
64.05398
[[1.5603281 ]
 [1.7205362 ]
 [0.96044827]]


ADAGRAD (ADAptive GRADient) is a variation of the SGD method in which the learning rate is adapted for each feature in the dataset (3 feature in this case): it assigns a higher learning rate to infrequent features, and a smaller learning rate to features that appears very frequently.

I don't think that in this case ADAGRAD would be helpful due to the small size of features in the dataset and the experimental fact that the plain SGD also performs very well.

In [110]:
beta = np.array([[1.0, 1.0, 1.0]]).T
delta = 1.0e-7
n_epochs = 2
learning_rate = 0.1
print(loss_jit(X, beta, y))
r = np.zeros(beta.shape)
for epoch in range(n_epochs):
    i = epoch % X.shape[0]

    grad = dloss_jit(X[i], beta, y[i])
    r += grad * grad
    beta -= learning_rate / (delta + np.sqrt(r)) * grad


print(loss_jit(X, beta, y))
print(beta)

110.0
89.49255
[[1.1624695]
 [1.1847999]
 [1.1371391]]


In [23]:
w_ls = np.linalg.pinv(X) @ y

print(w_ls)
print(loss(X, w_ls, y))

[[3.]
 [2.]
 [3.]]
5.364254e-29
